In [1]:
import requests
from bs4 import BeautifulSoup
import time

def get_questions(url):
    questions = []
    while len(questions) < 5000:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        question_links = soup.select('#site-container section div a')
        new_questions = [link.get_text(strip=True) for link in question_links if link.get_text(strip=True)]
        questions.extend(new_questions)
        time.sleep(2)

    return questions[:5000]

url = "https://www.answers.com/"
questions = get_questions(url)

# Print the questions
#for i, question in enumerate(questions, start=1):
    #print(f"Question {i}: {question}")

In [3]:
import mysql.connector

def load_questions(questions):
    conn = mysql.connector.connect(
        host='127.0.0.1',
        user='root', 
        passwd='Naresh@30701', 
        db='questions' 
    )
    cur = conn.cursor()
    
#creating table
    cur.execute("""
        CREATE TABLE IF NOT EXISTS Questions (
            id INT AUTO_INCREMENT PRIMARY KEY,
            question TEXT NOT NULL
        )
    """)

    for question in questions:
        cur.execute("INSERT INTO Questions (question) VALUES (%s)", (question,))
    conn.commit()
    cur.close()
    conn.close()

load_questions(questions)
